In [1]:
import datetime as dt
import pandas as pd
import os
import xml.etree.ElementTree as ET

In [3]:
%cd /content/drive/MyDrive/Colab_Weather_API/lib
import new_weather as nw
import weather_dict as wd
from importlib import reload

/content/drive/MyDrive/Colab_Weather_API/lib


In [4]:
%cd /content/drive/MyDrive/Colab_Weather_API/lib
reload(nw)
# reload(wd)

/content/drive/MyDrive/Colab_Weather_API/lib


<module 'new_weather' from '/content/drive/MyDrive/Colab_Weather_API/lib/new_weather.py'>

# serviceKey 생성

In [5]:
lib_path = "/content/drive/MyDrive/Colab_PRIVATE/weather_api"
service_key = nw.get_service_key(lib_path)

In [6]:
# API
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import xml.etree.ElementTree as ET

# API 호출

In [7]:
from datetime import datetime as dt

### 초단기실황, 초단기예보, 단기예보, 예보버전 정보를 조회하는 서비스입니다. 

* **초단기실황정보**는 예보 구역에 대한 대표 AWS 관측값을, 
* **초단기예보**는 예보시점부터 6시간 이내의 예보를, 
* **단기예보**는 예보기간과 구역을 시공간적으로 세분화한 예보를 제공합니다.

In [8]:
# sample URL
today_date = dt.now().strftime('%Y%m%d')
service_key = service_key

# 초단기실황조회
url_getUltraSrtNcst = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey={service_key}&numOfRows=10&pageNo=1&base_date={today_date}&base_time=0600&nx=55&ny=127'
# 초단기예보조회
url_getUltraSrtFcst = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst?serviceKey={service_key}&numOfRows=10&pageNo=1&base_date={today_date}&base_time=0630&nx=55&ny=127'
# 단기예보조회
url_getVilageFcst = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={service_key}&numOfRows=10&pageNo=1&base_date={today_date}&base_time=0500&nx=55&ny=127'
# 예보버전조회
url_getFcstVersion = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getFcstVersion?serviceKey={service_key}&numOfRows=10&pageNo=1&ftype=ODAM&basedatetime={today_date}0800'

In [9]:
# url_getFcstVersion

## 초단기실황조회

In [10]:
def getUltraSrtNcst(service_key:str, arg_dict:dict):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
    ServiceKey = service_key
    queryParams = '?' + urlencode({ 
                                   quote_plus('ServiceKey') : ServiceKey, 
                                   quote_plus('pageNo') : arg_dict['pageNo'], 
                                   quote_plus('numOfRows') : str(arg_dict['numOfRows']), 
                                   quote_plus('base_date') : arg_dict['base_date'], 
                                   quote_plus('base_time') : arg_dict['base_time'], 
                                   quote_plus('nx') : arg_dict['nx'], 
                                   quote_plus('ny') : arg_dict['ny'], 
                               
                                  })
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read()
    return response_body

In [11]:
arg_dict = \
{
    'numOfRows':'100',
    'pageNo':'1',
    'base_date':'20210720',
    'base_time':'0630',
    'nx':'55',
    'ny':'127'
 }

In [12]:
response_body = getUltraSrtNcst(service_key, arg_dict)
xtree = ET.fromstring(response_body)
nw.checker(xtree)

In [13]:
keys = ('baseDate', 'baseTime', 'category', 'nx', 'ny', 'obsrValue')

In [14]:
items = xtree.find('body').find('items').findall('item')
master = []
for item in items:
    sub_dict = {}
    for key in keys:
        sub_dict[key] = item.find(key).text
    master.append(sub_dict)

In [15]:
pd.DataFrame(master).sort_values(by=['category'])

,baseDate,baseTime,category,nx,ny,obsrValue
0,20210720,0600,PTY,55,127,0
1,20210720,0600,REH,55,127,91
2,20210720,0600,RN1,55,127,0
3,20210720,0600,T1H,55,127,24.6
4,20210720,0600,UUU,55,127,0
5,20210720,0600,VEC,55,127,20
6,20210720,0600,VVV,55,127,-0.1
7,20210720,0600,WSD,55,127,0.2


## 초단기예보조회

In [16]:
def getUltraSrtFcst(service_key:str, arg_dict:dict):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'
    ServiceKey = service_key
    queryParams = '?' + urlencode({ 
                                   quote_plus('ServiceKey') : ServiceKey, 
                                   quote_plus('pageNo') : arg_dict['pageNo'], 
                                   quote_plus('numOfRows') : str(arg_dict['numOfRows']), 
                                   quote_plus('base_date') : arg_dict['base_date'], 
                                   quote_plus('base_time') : arg_dict['base_time'], 
                                   quote_plus('nx') : arg_dict['nx'], 
                                   quote_plus('ny') : arg_dict['ny'], 
                               
                                  })
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read()
    return response_body

In [17]:
arg_dict = \
{
    'numOfRows':'100',
    'pageNo':'1',
    'base_date':'20210720',
    'base_time':'0630',
    'nx':'55',
    'ny':'127'
 }

In [18]:
response_body = getUltraSrtFcst(service_key, arg_dict)
xtree = ET.fromstring(response_body)
nw.checker(xtree)

In [19]:
keys = ('baseDate', 'baseTime', 'category', 'fcstDate', 'fcstTime', 'fcstValue', 'nx', 'ny')
# keys = ('category', 'fcstTime', 'fcstValue')

In [20]:
items = xtree.find('body').find('items').findall('item')
master = []
for item in items:
    sub_dict = {}
    for key in keys:
        sub_dict[key] = item.find(key).text
    master.append(sub_dict)

In [21]:
mdf = pd.DataFrame(master)
mdf
# mdf.sort_values(by=['fcstTime', 'category'])

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20210720,0630,LGT,20210720,0700,0,55,127
1,20210720,0630,LGT,20210720,0800,0,55,127
2,20210720,0630,LGT,20210720,0900,0,55,127
3,20210720,0630,LGT,20210720,1000,0,55,127
4,20210720,0630,LGT,20210720,1100,0,55,127
5,20210720,0630,LGT,20210720,1200,0,55,127
6,20210720,0630,PTY,20210720,0700,0,55,127
7,20210720,0630,PTY,20210720,0800,0,55,127
8,20210720,0630,PTY,20210720,0900,0,55,127
9,20210720,0630,PTY,20210720,1000,0,55,127


## 단기예보조회

In [22]:
def getVilageFcst(service_key:str, arg_dict:dict):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
    ServiceKey = service_key
    queryParams = '?' + urlencode({ 
                                   quote_plus('ServiceKey') : ServiceKey, 
                                   quote_plus('pageNo') : arg_dict['pageNo'], 
                                   quote_plus('numOfRows') : str(arg_dict['numOfRows']), 
                                   quote_plus('base_date') : arg_dict['base_date'], 
                                   quote_plus('base_time') : arg_dict['base_time'], 
                                   quote_plus('nx') : arg_dict['nx'], 
                                   quote_plus('ny') : arg_dict['ny'], 
                                  })
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    response_body = urlopen(request).read()
    return response_body

In [23]:
arg_dict = \
{
    'numOfRows':'1000',
    'pageNo':'1',
    'base_date':'20210720',
    'base_time':'0500',
    'nx':'55',
    'ny':'127'
 }

In [24]:
response_body = getVilageFcst(service_key, arg_dict)
xtree = ET.fromstring(response_body)
nw.checker(xtree)

In [25]:
keys = ('baseDate', 'baseTime', 'category', 'fcstDate', 'fcstTime', 'fcstValue', 'nx', 'ny')
# keys = ('category', 'fcstTime', 'fcstValue')

In [26]:
items = xtree.find('body').find('items').findall('item')
master = []
for item in items:
    sub_dict = {}
    for key in keys:
        sub_dict[key] = item.find(key).text
    master.append(sub_dict)

In [27]:
mdf = pd.DataFrame(master)
is_tmp = mdf['category'] == 'TMP'
mdf[is_tmp]
# mdf.sort_values(by=['fcstTime', 'category'])

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20210720,0500,TMP,20210720,0600,25,55,127
11,20210720,0500,TMP,20210720,0700,26,55,127
22,20210720,0500,TMP,20210720,0800,27,55,127
33,20210720,0500,TMP,20210720,0900,29,55,127
44,20210720,0500,TMP,20210720,1000,30,55,127
...,...,...,...,...,...,...,...,...
687,20210720,0500,TMP,20210722,2000,29,55,127
698,20210720,0500,TMP,20210722,2100,28,55,127
709,20210720,0500,TMP,20210722,2200,28,55,127
720,20210720,0500,TMP,20210722,2300,27,55,127


In [28]:
mdf['fcstTime'].unique()

array(['0600', '0700', '0800', '0900', '1000', '1100', '1200', '1300',
       '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100',
       '2200', '2300', '0000', '0100', '0200', '0300', '0400', '0500'],
      dtype=object)